In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy import stats
import math
import logging
from datetime import datetime

def setup_logging(output_dir):
    """Set up logging"""
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Create log filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file = os.path.join(output_dir, f"analysis_log_{timestamp}.log")
    
    # Configure logger
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file, encoding='utf-8'),
            logging.StreamHandler()  # Also output to console
        ]
    )
    
    logging.info(f"Logging initialized, log file: {log_file}")
    return log_file

def load_model_data(models, base_dir="./prediction_result_trafficmodel/"):
    """Load validation results for all models"""
    model_results = {}
    
    logging.info("Starting to load model data...")
    logging.info("-" * 60)
    logging.info(f"{'Model Name':<15} {'Status':<10} {'Records':<10}")
    logging.info("-" * 60)
    
    print("Model data loading status:")
    print("-" * 60)
    print(f"{'Model Name':<15} {'Status':<10} {'Records':<10}")
    print("-" * 60)
    
    for model in models:
        file_path = os.path.join(base_dir, model, "validation_results.csv")
        try:
            if os.path.exists(file_path):
                df = pd.read_csv(file_path)
                model_results[model] = df
                status_msg = f"{model:<15} {'Success':<10} {len(df):<10}"
                print(status_msg)
                logging.info(status_msg)
                logging.info(f"  Data preview: {df.head(2).to_dict()}")
            else:
                status_msg = f"{model:<15} {'File not found':<10} {0:<10}"
                print(status_msg)
                logging.warning(status_msg)
        except Exception as e:
            status_msg = f"{model:<15} {'Error':<10} {0:<10} - {str(e)}"
            print(status_msg)
            logging.error(status_msg)
    
    print("-" * 60)
    logging.info("-" * 60)
    
    # Log total data loaded
    total_records = sum(len(df) for df in model_results.values())
    logging.info(f"Loaded {len(model_results)} models, total records: {total_records}")
    
    return model_results

def calculate_metrics_with_stats(df):
    """Calculate evaluation metrics and their statistical properties"""
    if df is None or len(df) == 0:
        logging.warning("Received empty dataset for metrics calculation")
        return None
    
    logging.info(f"Calculating metrics, dataset size: {len(df)}")
    
    # Calculate errors
    errors = np.abs(df['predicted'] - df['actual'])
    squared_errors = np.square(df['predicted'] - df['actual'])
    
    # Calculate MAE and RMSE
    mae = np.mean(errors)
    rmse = np.sqrt(np.mean(squared_errors))
    
    # Calculate MAPE (handling zero actual values)
    mask = df['actual'] > 0
    if np.sum(mask) > 0:
        percentage_errors = np.abs((df['predicted'][mask] - df['actual'][mask]) / df['actual'][mask]) * 100
        mape = np.mean(percentage_errors)
        mape_var = np.var(percentage_errors, ddof=1)
        n_mape = sum(mask)
        logging.info(f"MAPE calculation: valid samples={n_mape}, mean={mape:.4f}, variance={mape_var:.4f}")
    else:
        mape = np.nan
        mape_var = np.nan
        n_mape = 0
        logging.warning("No valid MAPE samples (all actual values are 0)")
    
    # Calculate variance - fix MAE and RMSE variance calculation
    mae_var = np.var(errors, ddof=1)
    
    # Fix: Use Bootstrap method to estimate RMSE variance instead of direct calculation
    n = len(errors)
    bootstrap_rmse_samples = []
    for _ in range(500):  # 500 bootstrap samples
        indices = np.random.randint(0, n, size=n)
        bootstrap_squared_errors = squared_errors[indices]
        bootstrap_rmse = np.sqrt(np.mean(bootstrap_squared_errors))
        bootstrap_rmse_samples.append(bootstrap_rmse)
    
    rmse_var = np.var(bootstrap_rmse_samples, ddof=1)
    
    logging.info(f"MAE calculation: mean={mae:.4f}, variance={mae_var:.4f}")
    logging.info(f"RMSE calculation: mean={rmse:.4f}, variance={rmse_var:.4f}")
    
    # Sample size
    n = len(errors)
    
    # Calculate 95% confidence interval half-width (t-distribution)
    t_critical = stats.t.ppf(0.975, n-1)  # 95% confidence, two-tailed
    
    mae_ci_half = t_critical * np.sqrt(mae_var / n)
    rmse_ci_half = t_critical * np.sqrt(rmse_var / n)
    
    if n_mape > 1:
        mape_ci_half = t_critical * np.sqrt(mape_var / n_mape)
    else:
        mape_ci_half = np.nan
    
    logging.info(f"Confidence interval half-width: MAE={mae_ci_half:.4f}, RMSE={rmse_ci_half:.4f}, MAPE={f'{mape_ci_half:.4f}' if not np.isnan(mape_ci_half) else 'N/A'}")
    
    return {
        'MAE': mae,
        'MAE_var': mae_var,
        'MAE_ci_half': mae_ci_half,
        'RMSE': rmse,
        'RMSE_var': rmse_var,
        'RMSE_ci_half': rmse_ci_half,
        'MAPE': mape,
        'MAPE_var': mape_var,
        'MAPE_ci_half': mape_ci_half,
        'n_samples': n,
        'n_mape_samples': n_mape
    }

def bootstrap_ci(data, metric_function, n_bootstraps=2000, alpha=0.05):
    """Calculate confidence interval using bootstrap method"""
    bootstrap_samples = []
    n = len(data)
    
    logging.info(f"Starting Bootstrap analysis: samples={n}, repetitions={n_bootstraps}")
    
    for i in range(n_bootstraps):
        # Random sampling with replacement
        indices = np.random.randint(0, n, size=n)
        sample = data.iloc[indices]
        
        # Calculate statistic
        metric = metric_function(sample)
        bootstrap_samples.append(metric)
        
        # Log progress
        if i % 500 == 0 and i > 0:
            logging.info(f"  Bootstrap progress: {i}/{n_bootstraps} completed")
    
    # Calculate confidence interval
    ci_lower = np.percentile(bootstrap_samples, alpha/2 * 100)
    ci_upper = np.percentile(bootstrap_samples, (1-alpha/2) * 100)
    
    # Return mean value and confidence interval half-width
    mean_value = np.mean(bootstrap_samples)
    ci_half = (ci_upper - ci_lower) / 2
    
    logging.info(f"Bootstrap results: mean={mean_value:.6f}, confidence interval=[{ci_lower:.6f}, {ci_upper:.6f}], half-width={ci_half:.6f}")
    
    return mean_value, ci_half

def analyze_by_group(model_results, group_by):
    """Analyze by specified grouping variable"""
    group_metrics = {}
    
    logging.info(f"Starting analysis by '{group_by}' grouping...")
    
    for model, df in model_results.items():
        if group_by not in df.columns:
            logging.warning(f"Model {model} data does not have '{group_by}' column, skipping analysis")
            continue
            
        unique_values = df[group_by].unique()
        logging.info(f"Unique values of '{group_by}' in model {model}: {unique_values}")
        
        model_group_results = []
        for group_value in unique_values:
            group_df = df[df[group_by] == group_value]
            
            logging.info(f"Analyzing model {model}, {group_by}={group_value}, samples={len(group_df)}")
            
            metrics = calculate_metrics_with_stats(group_df)
            if metrics:
                metrics['model'] = model
                metrics[group_by] = group_value
                model_group_results.append(metrics)
        
        if model_group_results:
            group_metrics[model] = pd.DataFrame(model_group_results)
            logging.info(f"Completed analysis of model {model} by '{group_by}', {len(model_group_results)} groups")
    
    return group_metrics

def format_metrics_table(results_dict):
    """Format metrics table, adding ± symbols"""
    formatted_rows = []
    
    logging.info("Formatting metrics table...")
    
    for model, metrics in results_dict.items():
        # Standard format: value ± 95% confidence interval half-width
        mae_formatted = f"{metrics['MAE']:.4f} ± {metrics['MAE_ci_half']:.4f}"
        rmse_formatted = f"{metrics['RMSE']:.4f} ± {metrics['RMSE_ci_half']:.4f}"
        
        if not np.isnan(metrics['MAPE']):
            mape_formatted = f"{metrics['MAPE']:.2f} ± {metrics['MAPE_ci_half']:.2f}"
        else:
            mape_formatted = "N/A"
        
        row = {
            'Model': model,
            'MAE ± CI': mae_formatted,
            'RMSE ± CI': rmse_formatted,
            'MAPE(%) ± CI': mape_formatted,
            'Samples': metrics['n_samples'],
            'MAPE Samples': metrics['n_mape_samples']
        }
        
        logging.info(f"Formatted row: {row}")
        formatted_rows.append(row)
    
    return pd.DataFrame(formatted_rows)

def compare_bootstrap_vs_standard(overall_metrics, bootstrap_results):
    """Compare Bootstrap and standard confidence interval methods"""
    comparison_rows = []
    
    logging.info("Comparing Bootstrap and standard confidence interval methods...")
    
    for model in overall_metrics.keys():
        if model in bootstrap_results:
            std_metrics = overall_metrics[model]
            boot_metrics = bootstrap_results[model]
            
            comparison_rows.append({
                'Model': model,
                'MAE_standard': f"{std_metrics['MAE']:.4f} ± {std_metrics['MAE_ci_half']:.4f}",
                'MAE_bootstrap': f"{boot_metrics['MAE_bootstrap']:.4f} ± {boot_metrics['MAE_ci_bootstrap']:.4f}",
                'RMSE_standard': f"{std_metrics['RMSE']:.4f} ± {std_metrics['RMSE_ci_half']:.4f}",
                'RMSE_bootstrap': f"{boot_metrics['RMSE_bootstrap']:.4f} ± {boot_metrics['RMSE_ci_bootstrap']:.4f}",
                'MAPE_standard': f"{std_metrics['MAPE']:.2f} ± {std_metrics['MAPE_ci_half']:.2f}" if not np.isnan(std_metrics['MAPE']) else "N/A",
                'MAPE_bootstrap': f"{boot_metrics['MAPE_bootstrap']:.2f} ± {boot_metrics['MAPE_ci_bootstrap']:.2f}" if not np.isnan(boot_metrics['MAPE_bootstrap']) else "N/A"
            })
            
            logging.info(f"Model {model} comparison: "
                        f"MAE standard={std_metrics['MAE']:.4f}±{std_metrics['MAE_ci_half']:.4f} vs Bootstrap={boot_metrics['MAE_bootstrap']:.4f}±{boot_metrics['MAE_ci_bootstrap']:.4f}, "
                        f"RMSE standard={std_metrics['RMSE']:.4f}±{std_metrics['RMSE_ci_half']:.4f} vs Bootstrap={boot_metrics['RMSE_bootstrap']:.4f}±{boot_metrics['RMSE_ci_bootstrap']:.4f}")
    
    return pd.DataFrame(comparison_rows)

def main():
    # Model list
    models = ["LWR", "CTM", "METANET", "PI-LWR", "ML-CTM"]
    
    # Create output directory
    output_dir = "./model_metrics"
    os.makedirs(output_dir, exist_ok=True)
    
    # Set up logging
    log_file = setup_logging(output_dir)
    
    logging.info("=" * 80)
    logging.info("Starting traffic model performance metrics analysis")
    logging.info("=" * 80)
    
    print("Starting traffic model performance metrics analysis...\n")
    
    # 1. Load all model data
    model_results = load_model_data(models)
    
    if not model_results:
        logging.error("No valid model data found, analysis terminated")
        print("No valid model data found, analysis terminated")
        return
    
    # 2. Calculate overall performance metrics
    logging.info("\nCalculating overall model performance metrics...")
    print("\nCalculating overall model performance metrics...")
    overall_metrics = {}
    bootstrap_results = {}
    
    for model, df in model_results.items():
        logging.info(f"Analyzing overall performance of model {model}...")
        
        # Calculate standard metrics and statistics
        metrics = calculate_metrics_with_stats(df)
        if metrics:
            overall_metrics[model] = metrics
            
            # Additionally calculate confidence intervals using bootstrap
            logging.info(f"Using Bootstrap method to calculate confidence intervals for {model} model...")
            print(f"  Using Bootstrap method to calculate confidence intervals for {model} model...")
            
            # MAE bootstrap confidence interval
            mae_func = lambda x: np.mean(np.abs(x['predicted'] - x['actual']))
            mae_bootstrap, mae_ci = bootstrap_ci(df, mae_func)
            
            # RMSE bootstrap confidence interval
            rmse_func = lambda x: np.sqrt(np.mean(np.square(x['predicted'] - x['actual'])))
            rmse_bootstrap, rmse_ci = bootstrap_ci(df, rmse_func)
            
            # MAPE bootstrap confidence interval
            def mape_func(x):
                mask = x['actual'] > 0
                if sum(mask) > 0:
                    return np.mean(np.abs((x['predicted'][mask] - x['actual'][mask]) / x['actual'][mask])) * 100
                return np.nan
                
            mape_bootstrap, mape_ci = bootstrap_ci(df, mape_func)
            
            bootstrap_results[model] = {
                'MAE_bootstrap': mae_bootstrap,
                'MAE_ci_bootstrap': mae_ci,
                'RMSE_bootstrap': rmse_bootstrap,
                'RMSE_ci_bootstrap': rmse_ci,
                'MAPE_bootstrap': mape_bootstrap,
                'MAPE_ci_bootstrap': mape_ci
            }
    
    # 3. Format and save overall metrics results
    if overall_metrics:
        logging.info("Saving overall metrics results...")
        
        # Convert to DataFrame and save raw data
        overall_df = pd.DataFrame(overall_metrics).T.reset_index().rename(columns={'index': 'model'})
        overall_file = os.path.join(output_dir, "overall_metrics_raw.csv")
        overall_df.to_csv(overall_file, index=False)
        logging.info(f"Saved raw metrics to: {overall_file}")
        
        # Add bootstrap results
        if bootstrap_results:
            bootstrap_df = pd.DataFrame(bootstrap_results).T.reset_index().rename(columns={'index': 'model'})
            bootstrap_file = os.path.join(output_dir, "bootstrap_metrics.csv")
            bootstrap_df.to_csv(bootstrap_file, index=False)
            logging.info(f"Saved Bootstrap metrics to: {bootstrap_file}")
            
            # Compare results from both methods
            comparison_df = compare_bootstrap_vs_standard(overall_metrics, bootstrap_results)
            comparison_file = os.path.join(output_dir, "metrics_method_comparison.csv")
            comparison_df.to_csv(comparison_file, index=False)
            logging.info(f"Saved method comparison to: {comparison_file}")
        
        # Create formatted table
        formatted_df = format_metrics_table(overall_metrics)
        formatted_file = os.path.join(output_dir, "overall_metrics_formatted.csv") 
        formatted_df.to_csv(formatted_file, index=False)
        logging.info(f"Saved formatted metrics to: {formatted_file}")
        
        # Create another formatted table using bootstrap results
        if bootstrap_results:
            bootstrap_formatted_rows = []
            for model, metrics in bootstrap_results.items():
                std_metrics = overall_metrics[model]
                bootstrap_formatted_rows.append({
                    'Model': model,
                    'MAE ± CI': f"{metrics['MAE_bootstrap']:.4f} ± {metrics['MAE_ci_bootstrap']:.4f}",
                    'RMSE ± CI': f"{metrics['RMSE_bootstrap']:.4f} ± {metrics['RMSE_ci_bootstrap']:.4f}",
                    'MAPE(%) ± CI': f"{metrics['MAPE_bootstrap']:.2f} ± {metrics['MAPE_ci_bootstrap']:.2f}" if not np.isnan(metrics['MAPE_bootstrap']) else "N/A",
                    'Samples': std_metrics['n_samples'],
                    'MAPE Samples': std_metrics['n_mape_samples']
                })
            
            bootstrap_formatted_df = pd.DataFrame(bootstrap_formatted_rows)
            bootstrap_formatted_file = os.path.join(output_dir, "overall_metrics_bootstrap_formatted.csv")
            bootstrap_formatted_df.to_csv(bootstrap_formatted_file, index=False)
            logging.info(f"Saved Bootstrap formatted metrics to: {bootstrap_formatted_file}")
        
        # Print formatted table
        print("\nOverall performance metrics (95% confidence interval):")
        print(formatted_df.to_string(index=False))
        
        logging.info("\nOverall performance metrics (95% confidence interval):")
        logging.info("\n" + formatted_df.to_string(index=False))
    
    # 4. Analyze by prediction time
    logging.info("\nAnalyzing by prediction time...")
    print("\nAnalyzing by prediction time...")
    time_group_metrics = analyze_by_group(model_results, 'predict_minutes')
    
    if time_group_metrics:
        # Combine results from all models
        combined_df = pd.concat(time_group_metrics.values())
        combined_file = os.path.join(output_dir, "metrics_by_predict_time.csv")
        combined_df.to_csv(combined_file, index=False)
        logging.info(f"Saved metrics by prediction time to: {combined_file}")
        
        # Create formatted version
        formatted_rows = []
        for model, df in time_group_metrics.items():
            for _, row in df.iterrows():
                formatted_rows.append({
                    'Model': model,
                    'Predict_Minutes': row['predict_minutes'],
                    'MAE ± CI': f"{row['MAE']:.4f} ± {row['MAE_ci_half']:.4f}",
                    'RMSE ± CI': f"{row['RMSE']:.4f} ± {row['RMSE_ci_half']:.4f}",
                    'MAPE(%) ± CI': f"{row['MAPE']:.2f} ± {row['MAPE_ci_half']:.2f}" if not np.isnan(row['MAPE']) else "N/A"
                })
        
        formatted_time_df = pd.DataFrame(formatted_rows)
        formatted_time_file = os.path.join(output_dir, "formatted_metrics_by_predict_time.csv")
        formatted_time_df.to_csv(formatted_time_file, index=False)
        logging.info(f"Saved formatted metrics by prediction time to: {formatted_time_file}")
    
    # 5. Analyze by vehicle type
    logging.info("Analyzing by vehicle type...")
    print("Analyzing by vehicle type...")
    vehicle_group_metrics = analyze_by_group(model_results, 'vehicle_type')
    
    if vehicle_group_metrics:
        # Combine results from all models
        combined_df = pd.concat(vehicle_group_metrics.values())
        combined_file = os.path.join(output_dir, "metrics_by_vehicle_type.csv")
        combined_df.to_csv(combined_file, index=False)
        logging.info(f"Saved metrics by vehicle type to: {combined_file}")
        
        # Create formatted version
        formatted_rows = []
        for model, df in vehicle_group_metrics.items():
            for _, row in df.iterrows():
                formatted_rows.append({
                    'Model': model,
                    'Vehicle_Type': row['vehicle_type'],
                    'MAE ± CI': f"{row['MAE']:.4f} ± {row['MAE_ci_half']:.4f}",
                    'RMSE ± CI': f"{row['RMSE']:.4f} ± {row['RMSE_ci_half']:.4f}",
                    'MAPE(%) ± CI': f"{row['MAPE']:.2f} ± {row['MAPE_ci_half']:.2f}" if not np.isnan(row['MAPE']) else "N/A"
                })
        
        formatted_vehicle_df = pd.DataFrame(formatted_rows)
        formatted_vehicle_file = os.path.join(output_dir, "formatted_metrics_by_vehicle_type.csv")
        formatted_vehicle_df.to_csv(formatted_vehicle_file, index=False)
        logging.info(f"Saved formatted metrics by vehicle type to: {formatted_vehicle_file}")
    
    # 6. Analyze by segment type (if the column exists in the data)
    has_segment_type = any('segment_type' in df.columns for df in model_results.values())
    logging.info(f"segment_type column exists in data: {has_segment_type}")
    
    if has_segment_type:
        logging.info("Analyzing by segment type...")
        print("Analyzing by segment type...")
        segment_group_metrics = analyze_by_group(model_results, 'segment_type')
        
        if segment_group_metrics:
            # Combine results from all models
            combined_df = pd.concat(segment_group_metrics.values())
            combined_file = os.path.join(output_dir, "metrics_by_segment_type.csv")
            combined_df.to_csv(combined_file, index=False)
            logging.info(f"Saved metrics by segment type to: {combined_file}")
            
            # Create formatted version
            formatted_rows = []
            for model, df in segment_group_metrics.items():
                for _, row in df.iterrows():
                    formatted_rows.append({
                        'Model': model,
                        'Segment_Type': row['segment_type'],
                        'MAE ± CI': f"{row['MAE']:.4f} ± {row['MAE_ci_half']:.4f}",
                        'RMSE ± CI': f"{row['RMSE']:.4f} ± {row['RMSE_ci_half']:.4f}",
                        'MAPE(%) ± CI': f"{row['MAPE']:.2f} ± {row['MAPE_ci_half']:.2f}" if not np.isnan(row['MAPE']) else "N/A"
                    })
            
            formatted_segment_df = pd.DataFrame(formatted_rows)
            formatted_segment_file = os.path.join(output_dir, "formatted_metrics_by_segment_type.csv")
            formatted_segment_df.to_csv(formatted_segment_file, index=False)
            logging.info(f"Saved formatted metrics by segment type to: {formatted_segment_file}")
    
    # 7. Create a results summary file
    summary_msg = f"""
Analysis Results Summary
===================
Analysis completed at: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
Models analyzed: {', '.join(model_results.keys())}
Total samples: {sum(len(df) for df in model_results.values())}
Output directory: {output_dir}
Log file: {log_file}

Files generated:
{os.linesep.join([f"- {file}" for file in os.listdir(output_dir) if file.endswith('.csv')])}

Notes:
- Confidence intervals were calculated using both standard method (t-distribution) and Bootstrap method
- Bootstrap method typically provides more accurate confidence interval estimates, especially for non-normal data
- Please refer to the CSV files and log file for detailed results
"""

    summary_file = os.path.join(output_dir, "analysis_summary.txt")
    with open(summary_file, 'w', encoding='utf-8') as f:
        f.write(summary_msg)
    
    logging.info("\n" + summary_msg)
    print(f"\nAnalysis complete! All results have been saved to: {output_dir}")
    print(f"Log file: {log_file}")

if __name__ == "__main__":
    main()